### 웹툰 메인 이미지 스크래핑
* 추천 웹툰의 title과 link 가져오기
* {title:'대만에서, 너와 하루', link:'https://comic.naver.com/webtoon/list.nhn?titleId=749279'}
* dict 한개가 웹툰 한 개 정보
* 근데 추천웹툰이 여러개니까 그걸 또 list에 담을 것.
* [{title:' '. link:' '}{}{}] 뭐 이런식이요


In [199]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

main_url = 'https://comic.naver.com/index.nhn'

res = requests.get(main_url) # url들을 가져오는거지
html = res.text # 페이지 소스보기 해서 보이는것들 싹 가져오는거
# 이제 beautifulsoup 을 이용해서 객체를 생성해서 parsing하겠다고 해야돼
soup = BeautifulSoup(html, 'html.parser')

# 내가 원하는 부분 ul tag의 id를 가져올거야 그건 중복되지않는 문자를 쓸거거든
# copy selector 했을때 나오는 이 좌표 키워드들 '#genreRecommand > li:nth-child(2) > div.genreRecomImg2 > a' 참조해
print(type(soup.select('#genreRecommand'))) # ul태그라고 안쓰고 대신 '#'붙임
print(len(soup.select('#genreRecommand li')))
# 여기까지 하면 지금 추천웹툰 gsnreRecommand가 6개 있기때문에 li가 6개있는게 맞아용
print(len(soup.select('#content ul li a')))
# 여기까지 하면 이미지, 제목, 회차설명, 작가명에 모두 a태그가 붙어있기 때문에 24가 뜸
# <ul> -자식-> <li> -> <a> 인데 범주로 생각해보면 ul전체추천웹툰 li개수 a밑에 항목
# /webtoon/list.nhn?titleId=749279

print(len(soup.select('#genreRecommand h6 a')))

webtoon_list = []

for a_tag in soup.select('#genreRecommand h6 a'):
    title = a_tag.text.strip()
    link = urljoin(main_url, a_tag['href']) # 속성에는 [] 준다구요
    print(title, '-',link)
    webtoon_dict = {'title':title, 'link':link}

    webtoon_list.append(webtoon_dict)
    webtoon_dict = {}
    print()

print(webtoon_list)

<class 'list'>
6
89
6
히어로메이커 - https://comic.naver.com/webtoon/list.nhn?titleId=21815

대만에서, 너와 하루 - https://comic.naver.com/webtoon/list.nhn?titleId=749279

야자괴담 - https://comic.naver.com/webtoon/list.nhn?titleId=743025

중독연구소 - https://comic.naver.com/webtoon/list.nhn?titleId=740132

결혼생활 그림일기 - https://comic.naver.com/webtoon/list.nhn?titleId=741891

귀전구담 - https://comic.naver.com/webtoon/list.nhn?titleId=694946

[{'title': '히어로메이커', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=21815'}, {'title': '대만에서, 너와 하루', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=749279'}, {'title': '야자괴담', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=743025'}, {'title': '중독연구소', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=740132'}, {'title': '결혼생활 그림일기', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=741891'}, {'title': '귀전구담', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=694946'}]


### 특정 웹툰의 image 다운로드를 함수로 선언하기

In [200]:
def write_image(title, main_url):
    print(title, main_url)
    # 이거 밑에꺼 앞에 쌤이 주신거에서 복붙해온거요
    import requests
    from bs4 import BeautifulSoup
    import os

    res = requests.get(main_url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')

    img_urls = []
    for img_url in soup.select("img[src$='.jpg']"):
        #print(img_url)
        img_urls.append(img_url['src'])

    # 디렉토리 생성
    dir_name = 'img/' + title
    if not os.path.isdir(dir_name): # img밑에 받은 title 명으로 폴더만들겠다는 뜻
        os.mkdir(dir_name)

    print(len(img_urls)) 
    for img_url in img_urls:
        #print(img_url)
        req_header = {
            'referer':main_url
        }

        res2 = requests.get(img_url, headers=req_header)
        img_data = res2.content
        file_name = os.path.basename(img_url)
        with open(dir_name + '/' + file_name,'wb') as file:
            print('Writing to {} ({} bytes)'.format(file_name,len(img_data)))    
            file.write(img_data)    

In [201]:
write_image('대만에서, 너와 하루', 'https://comic.naver.com/webtoon/list.nhn?titleId=749279')

대만에서, 너와 하루 https://comic.naver.com/webtoon/list.nhn?titleId=749279
13
Writing to thumbnail_IMAG06_0862a825-7c18-49ac-ba0c-9ac0f454f11c.jpg (16593 bytes)
Writing to thumbnail_202x120_62e257c6-32ac-4159-80a9-2790cbd7d5cd.jpg (27051 bytes)
Writing to thumbnail_202x120_9c838080-555d-456d-8698-094d0c49b8d0.jpg (32048 bytes)
Writing to thumbnail_202x120_460b61a7-b8ef-4be8-964d-c867dcab62a5.jpg (32975 bytes)
Writing to thumbnail_202x120_156deca2-f458-441b-92ce-cb4250d352fc.jpg (28466 bytes)
Writing to thumbnail_IMAG10_85425026-fd1c-4a8a-8d58-e257bbf892c0.jpg (22743 bytes)
Writing to thumbnail_IMAG10_545c2365-1d18-4cd3-afd9-27b161a4c664.jpg (12749 bytes)
Writing to thumbnail_IMAG10_b324ec4f-f869-4b55-b2fb-63306bef3a41.jpg (10601 bytes)
Writing to thumbnail_IMAG10_69a59ea4-ece8-4454-beea-a730adae236f.jpg (10347 bytes)
Writing to thumbnail_IMAG10_1ffbcabb-b5fd-41d8-9500-faaee1d3cbb4.jpg (9410 bytes)
Writing to bnr_partnership.jpg (10326 bytes)
Writing to bnr_joinus.jpg (17502 bytes)
Writing to 

In [99]:
import requests
from bs4 import BeautifulSoup
import os

main_url = 'https://comic.naver.com/index.nhn'

res = requests.get(main_url)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

print(len(soup.select('#genreRecommand div.genreRecomInfo2 h6 a')))

haeboja = soup.select('#genreRecommand div.genreRecomInfo2 h6 a')



for img_url in soup.select('#genreRecommand div.genreRecomInfo2 h6 a'):
    img_urls.append(img_url['href'])
    
    
img_urls

6


['/webtoon/list.nhn?titleId=626906',
 '/webtoon/list.nhn?titleId=20853',
 '/webtoon/list.nhn?titleId=616239',
 '/webtoon/list.nhn?titleId=716857',
 '/webtoon/list.nhn?titleId=703850',
 '/webtoon/list.nhn?titleId=743139',
 '/webtoon/list.nhn?titleId=21815',
 '/webtoon/list.nhn?titleId=749279',
 '/webtoon/list.nhn?titleId=743025',
 '/webtoon/list.nhn?titleId=740132',
 '/webtoon/list.nhn?titleId=741891',
 '/webtoon/list.nhn?titleId=694946']

### 쌤이하심

In [210]:
import requests
from bs4 import BeautifulSoup
import os

main_url = 'https://comic.naver.com/index.nhn'

res = requests.get(main_url)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

webtoon_list = []
webtoon_dict = {}

for div_tag in soup.select(' .genreRecomInfo2'):
    # for a_tag1 in div_tag.select('h6 a'): 이래도 되고
    a_tag1 = div_tag.select('h6 a')[0]
    title = a_tag1.text.strip()
    link = urljoin(main_url, a_tag1['href'])
    webtoon_dict["title"] = title
    webtoon_dict["link"] = link
        
    for a_tag2 in div_tag.select('p a'):
    # a_tag2 = div_tag.select('p a')[0] 이래도 돼
        detail_link = urljoin(main_url, a_tag2['href'])
        print(detail_link)
        webtoon_dict["detail_link"] = detail_link
    webtoon_list.append(webtoon_dict)
    webtoon_dict = {}
    
webtoon_list

https://comic.naver.com/webtoon/detail.nhn?titleId=400739&no=388
https://comic.naver.com/webtoon/detail.nhn?titleId=731019&no=43
https://comic.naver.com/webtoon/detail.nhn?titleId=746535&no=3
https://comic.naver.com/webtoon/detail.nhn?titleId=731063&no=53
https://comic.naver.com/webtoon/detail.nhn?titleId=730656&no=55
https://comic.naver.com/webtoon/detail.nhn?titleId=733080&no=44


[{'title': '에이머',
  'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=400739',
  'detail_link': 'https://comic.naver.com/webtoon/detail.nhn?titleId=400739&no=388'},
 {'title': '그날 죽은 나는',
  'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=731019',
  'detail_link': 'https://comic.naver.com/webtoon/detail.nhn?titleId=731019&no=43'},
 {'title': '언메이크',
  'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=746535',
  'detail_link': 'https://comic.naver.com/webtoon/detail.nhn?titleId=746535&no=3'},
 {'title': '킬링타임',
  'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=731063',
  'detail_link': 'https://comic.naver.com/webtoon/detail.nhn?titleId=731063&no=53'},
 {'title': '사신소년',
  'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=730656',
  'detail_link': 'https://comic.naver.com/webtoon/detail.nhn?titleId=730656&no=55'},
 {'title': '그놈은 흑염룡',
  'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=733080',
  'detail_link': 'https://comic.naver.com

In [211]:
for idx, webtoon in enumerate(webtoon_list):
    title = webtoon['title']
    detail_link = webtoon['link']
    print('-----', idx, title, detail_link)
    write_image(title, detail_link)

----- 0 에이머 https://comic.naver.com/webtoon/list.nhn?titleId=400739
에이머 https://comic.naver.com/webtoon/list.nhn?titleId=400739
19
Writing to thumbnail_IMAG06_7ef2edff-f2fa-494c-ab2d-ce67d1324ad7.jpg (152447 bytes)
Writing to thumbnail_202x120_66895b7c-c296-4c4f-ac1d-3d6b0284a15c.jpg (15324 bytes)
Writing to thumbnail_202x120_f2020087-7da6-4637-8a27-436bbbde5390.jpg (17431 bytes)
Writing to thumbnail_202x120_a732535f-da9d-4329-a191-488ce614924c.jpg (36165 bytes)
Writing to thumbnail_202x120_a802c203-d433-4c0d-b71d-a546a810b333.jpg (21921 bytes)
Writing to thumbnail_202x120_3397b724-e66c-47dc-b4aa-f2318247dd46.jpg (29075 bytes)
Writing to thumbnail_202x120_4ef53f09-4d9f-4469-a791-3a75c7dc1e03.jpg (32304 bytes)
Writing to thumbnail_202x120_9d3e1ccf-c0aa-4ea1-9d58-212b520c2293.jpg (35492 bytes)
Writing to thumbnail_202x120_db32761b-ea9c-422e-8863-f461ba7fb67d.jpg (21310 bytes)
Writing to thumbnail_202x120_252643d3-daab-455e-9b14-0440237d9a52.jpg (33427 bytes)
Writing to thumbnail_202x120_

Writing to thumbnail_202x120_0af9d508-1ca3-4288-ab89-2cff14919f7d.jpg (31744 bytes)
Writing to thumbnail_IMAG10_85425026-fd1c-4a8a-8d58-e257bbf892c0.jpg (22743 bytes)
Writing to thumbnail_IMAG10_545c2365-1d18-4cd3-afd9-27b161a4c664.jpg (12749 bytes)
Writing to thumbnail_IMAG10_b324ec4f-f869-4b55-b2fb-63306bef3a41.jpg (10601 bytes)
Writing to thumbnail_IMAG10_69a59ea4-ece8-4454-beea-a730adae236f.jpg (10347 bytes)
Writing to thumbnail_IMAG10_1ffbcabb-b5fd-41d8-9500-faaee1d3cbb4.jpg (9410 bytes)
Writing to bnr_partnership.jpg (10326 bytes)
Writing to bnr_joinus.jpg (17502 bytes)
Writing to bnr_careers.jpg (18455 bytes)
